In [3]:
# Advanced Bike-Sharing Demand — Research-Grade End-to-End (Colab)
# Paste entire cell into Colab and run.
# Features: robust FE, lag features, TimeSeries CV, RandomizedSearch, HistGradientBoostingRegressor, permutation importance.

import os, io, zipfile, urllib.request, json, warnings, math
from dataclasses import dataclass
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
from sklearn.base import TransformerMixin, BaseEstimator

import joblib
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

# ---------------------------
# Config
# ---------------------------
DATASET = "hour"   # "hour" or "day"
TEST_SIZE = 0.2    # chronological holdout
RANDOM_STATE = 42
ARTIFACTS = "artifacts"
USE_LOG_TARGET = False   # set True to model log1p(cnt)
LAGS = [1, 24]           # lag features to add (1, 24 hours)
ROLL_WINDOWS = [3, 24]   # rolling windows (in hours) if hour data

np.random.seed(RANDOM_STATE)

# ---------------------------
# Utilities & Eval dataclass
# ---------------------------
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def rmse(y, yhat):
    return math.sqrt(mean_squared_error(y, yhat))

@dataclass
class EvalResult:
    name: str
    train_rmse: float
    train_mae: float
    train_r2: float
    test_rmse: float
    test_mae: float
    test_r2: float

    def as_dict(self):
        return {
            "train_rmse": self.train_rmse,
            "train_mae": self.train_mae,
            "train_r2": self.train_r2,
            "test_rmse": self.test_rmse,
            "test_mae": self.test_mae,
            "test_r2": self.test_r2,
        }

# ---------------------------
# Data download / load
# ---------------------------
UCI_ZIP = "https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip"
HOUR_CSV = "hour.csv"
DAY_CSV = "day.csv"

def download_and_extract():
    try:
        print("Downloading UCI dataset...")
        with urllib.request.urlopen(UCI_ZIP, timeout=60) as r:
            data = r.read()
        z = zipfile.ZipFile(io.BytesIO(data))
        for fn in [HOUR_CSV, DAY_CSV]:
            if fn in z.namelist():
                print(f"Extracting {fn}")
                with z.open(fn) as src, open(fn, "wb") as dst:
                    dst.write(src.read())
        print("Done.")
    except Exception as e:
        print("Download failed:", e)

def load_dataset(kind="hour"):
    assert kind in ("hour","day")
    target = HOUR_CSV if kind=="hour" else DAY_CSV
    if not os.path.exists(target):
        download_and_extract()
    if os.path.exists(target):
        df = pd.read_csv(target)
        print(f"Loaded {target} shape={df.shape}")
        return df
    # fallback synthetic
    print("Falling back to synthetic dataset (for demo).")
    if kind=="hour":
        dates = pd.date_range("2018-01-01", periods=24*365, freq="H")
        n = len(dates)
        df = pd.DataFrame({
            "dteday": dates.date,
            "hr": dates.hour,
            "weekday": dates.weekday,
            "mnth": dates.month,
            "season": ((dates.month % 12) // 3) + 1,
            "workingday": ((dates.weekday < 5) * 1),
            "holiday": 0,
            "weathersit": np.random.choice([1,2,3], size=n, p=[0.7,0.25,0.05]),
            "temp": np.clip(0.5 + 0.4*np.sin(2*np.pi*dates.dayofyear/365 - np.pi/2) + np.random.normal(0,0.05,n), 0, 1),
            "atemp": np.random.rand(n),
            "hum": np.clip(0.6 + 0.2*np.sin(2*np.pi*dates.dayofyear/365) + np.random.normal(0,0.1,n), 0, 1),
            "windspeed": np.clip(0.3 + np.random.normal(0,0.1,n), 0, 1),
        })
        # create cnt with peaks
        hour = df["hr"].values
        commute = 80*np.exp(-0.5*((hour-8)/2.2)**2) + 90*np.exp(-0.5*((hour-18)/2.5)**2)
        base = 120 + 200*df["workingday"]
        noise = np.random.normal(0,40,len(df))
        df["cnt"] = np.clip(base + commute + 100*(df["temp"]-0.3) + noise, 0, None).astype(int)
        return df
    else:
        dates = pd.date_range("2018-01-01", periods=730, freq="D")
        n = len(dates)
        df = pd.DataFrame({
            "dteday": dates.date,
            "weekday": dates.weekday,
            "mnth": dates.month,
            "season": ((dates.month % 12) // 3) + 1,
            "workingday": ((dates.weekday < 5) * 1),
            "holiday": 0,
            "weathersit": np.random.choice([1,2,3], size=n, p=[0.7,0.25,0.05]),
            "temp": np.clip(0.5 + 0.4*np.sin(2*np.pi*dates.dayofyear/365 - np.pi/2) + np.random.normal(0,0.05,n), 0, 1),
            "atemp": np.random.rand(n),
            "hum": np.clip(0.6 + 0.2*np.sin(2*np.pi*dates.dayofyear/365) + np.random.normal(0,0.1,n), 0, 1),
            "windspeed": np.clip(0.3 + np.random.normal(0,0.1,n), 0, 1),
        })
        base = 2500 + 800*df["workingday"]
        noise = np.random.normal(0,300,n)
        df["cnt"] = np.clip(base + 1200*(df["season"].isin([2,3])) + 1500*(df["temp"]-0.3) + noise, 0, None).astype(int)
        return df

# ---------------------------
# Feature engineering transformer (create lags inside pipeline isn't trivial,
# so we create lags before pipeline as they require access to ordered rows)
# ---------------------------
def create_time_features(df, kind="hour"):
    df = df.copy()
    if "dteday" in df.columns:
        df["dteday"] = pd.to_datetime(df["dteday"])
    if kind=="hour":
        # ensure hr exists (hourly dataset)
        if "hr" not in df.columns:
            # maybe hour provided via datetime index
            df["hr"] = df["dteday"].dt.hour
        df["timestamp"] = df["dteday"] + pd.to_timedelta(df["hr"], unit="h")
    else:
        df["timestamp"] = pd.to_datetime(df["dteday"])
    df = df.sort_values("timestamp").reset_index(drop=True)
    # cyclical encodings
    if "hr" in df.columns:
        df["hr_sin"] = np.sin(2*np.pi*df["hr"]/24)
        df["hr_cos"] = np.cos(2*np.pi*df["hr"]/24)
    doy = df["timestamp"].dt.dayofyear
    df["doy_sin"] = np.sin(2*np.pi*doy/365)
    df["doy_cos"] = np.cos(2*np.pi*doy/365)
    df["is_weekend"] = df["weekday"].isin([5,6]).astype(int) if "weekday" in df.columns else 0
    # drop leakage
    for c in ["casual","registered","instant"]:
        if c in df.columns:
            df.drop(columns=[c], inplace=True)
    return df

def add_lag_and_roll_features(df, lags=[1,24], roll_windows=[3,24], target="cnt"):
    df = df.copy()
    for lag in lags:
        df[f"lag_{lag}"] = df[target].shift(lag)
    for w in roll_windows:
        df[f"roll_mean_{w}"] = df[target].shift(1).rolling(window=w, min_periods=1).mean()
    # After lag/roll creation we drop rows with NaN (start of series)
    df = df.dropna().reset_index(drop=True)
    return df

# ---------------------------
# Preprocessing & columns
# ---------------------------
def get_feature_lists(df):
    numeric_candidates = ["temp","atemp","hum","windspeed","hr_sin","hr_cos","doy_sin","doy_cos"] + [f"lag_{l}" for l in LAGS] + [f"roll_mean_{w}" for w in ROLL_WINDOWS]
    numeric = [c for c in numeric_candidates if c in df.columns]
    categorical_candidates = ["season","mnth","hr","weekday","workingday","holiday","weathersit","yr","is_weekend","is_peak_hour"]
    categorical = [c for c in categorical_candidates if c in df.columns]
    return numeric, categorical

def build_preprocessor(numeric, categorical):
    num_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])

    pre = ColumnTransformer([
        ("num", num_pipe, numeric),
        ("cat", cat_pipe, categorical)
    ], remainder="drop")
    return pre

# ---------------------------
# Modeling & tuning
# ---------------------------
def fit_and_tune(X_train, y_train, pre):
    # Models to tune (Ridge/Lasso/HistGB)
    estimators = {
        "Ridge": Ridge(random_state=RANDOM_STATE),
        "Lasso": Lasso(random_state=RANDOM_STATE, max_iter=5000),
        "HistGB": HistGradientBoostingRegressor(random_state=RANDOM_STATE)
    }

    # Pipelines
    pipes = {k: Pipeline([("pre", pre), ("model", v)]) for k,v in estimators.items()}

    # Hyperparameter search spaces (Randomized)
    param_distributions = {
        "Ridge": {
            "model__alpha": np.logspace(-3,3,50)
        },
        "Lasso": {
            "model__alpha": np.logspace(-4,0,50)
        },
        "HistGB": {
            "model__learning_rate": [0.01,0.03,0.05,0.1],
            "model__max_iter": [100,200,400],
            "model__max_leaf_nodes": [15,31,63,127],
            "model__min_samples_leaf": [5,10,20]
        }
    }

    tscv = TimeSeriesSplit(n_splits=5)
    fitted = {}
    results = {}

    for name, pipe in pipes.items():
        print(f"\nTuning {name} ...")
        params = param_distributions.get(name, {})
        if params:
            search = RandomizedSearchCV(pipe, params, n_iter=30, cv=tscv, scoring="neg_root_mean_squared_error", n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
            search.fit(X_train, y_train)
            best = search.best_estimator_
            print(f" -> best params: {search.best_params_}")
        else:
            pipe.fit(X_train, y_train)
            best = pipe
        # compute train predictions (on training portion) and store
        fitted[name] = best
    return fitted

# ---------------------------
# Evaluation helpers
# ---------------------------
def evaluate_model(pipe, Xtr, ytr, Xte, yte, use_log=False):
    # predictions
    if use_log:
        yp_tr = np.expm1(pipe.predict(Xtr))
        yp_te = np.expm1(pipe.predict(Xte))
    else:
        yp_tr = pipe.predict(Xtr)
        yp_te = pipe.predict(Xte)
    res = EvalResult(
        name = getattr(pipe.named_steps["model"], "__class__", type(pipe)).__name__,
        train_rmse = rmse(ytr, yp_tr),
        train_mae = mean_absolute_error(ytr, yp_tr),
        train_r2 = r2_score(ytr, yp_tr),
        test_rmse = rmse(yte, yp_te),
        test_mae = mean_absolute_error(yte, yp_te),
        test_r2 = r2_score(yte, yp_te),
    )
    return res, yp_tr, yp_te

# ---------------------------
# Plotting functions (EDA + diagnostics)
# ---------------------------
def eda_plots(df):
    ensure_dir(ARTIFACTS)
    print("\n--- EDA plots saving to artifacts/ ---")
    plt.figure(figsize=(8,5))
    sns.histplot(df['cnt'], bins=40, kde=True)
    plt.title("Distribution of bike rentals (cnt)")
    plt.savefig(os.path.join(ARTIFACTS, "eda_cnt_dist.png")); plt.close()

    if "hr" in df.columns:
        plt.figure(figsize=(10,6))
        df_group = df.groupby("hr")["cnt"].mean().reset_index()
        sns.lineplot(data=df_group, x="hr", y="cnt")
        plt.title("Average rentals by hour")
        plt.savefig(os.path.join(ARTIFACTS, "eda_avg_by_hour.png")); plt.close()

    plt.figure(figsize=(8,6))
    sns.boxplot(x="season", y="cnt", data=df)
    plt.title("Rentals by season")
    plt.savefig(os.path.join(ARTIFACTS, "eda_by_season.png")); plt.close()

    plt.figure(figsize=(6,5))
    sns.barplot(x="workingday", y="cnt", data=df, estimator=np.mean)
    plt.title("Average rentals: workingday vs non-workingday")
    plt.savefig(os.path.join(ARTIFACTS, "eda_workingday.png")); plt.close()

    plt.figure(figsize=(10,8))
    sns.heatmap(df.corr(), cmap="coolwarm", center=0)
    plt.title("Correlation heatmap")
    plt.savefig(os.path.join(ARTIFACTS, "eda_corr.png")); plt.close()
    print("EDA plots saved.")

def plot_predictions_and_residuals(ts_test, y_test, y_pred, model_name):
    ensure_dir(ARTIFACTS)
    # scatter
    plt.figure(figsize=(7,6))
    plt.scatter(y_test, y_pred, alpha=0.4)
    mx = max(y_test.max(), np.nanmax(y_pred))
    plt.plot([0,mx],[0,mx],'r--')
    plt.xlabel("True cnt"); plt.ylabel("Predicted cnt"); plt.title(f"{model_name} True vs Predicted (scatter)")
    plt.savefig(os.path.join(ARTIFACTS, f"{model_name}_true_vs_pred_scatter.png")); plt.close()

    # residual histogram
    resid = y_test - y_pred
    plt.figure(figsize=(7,5))
    sns.histplot(resid, bins=40, kde=True)
    plt.title(f"{model_name} Residual distribution")
    plt.savefig(os.path.join(ARTIFACTS, f"{model_name}_resid_hist.png")); plt.close()

    # time series sample (first 500 points)
    nplot = min(500, len(y_test))
    plt.figure(figsize=(12,5))
    plt.plot(ts_test.iloc[:nplot], y_test[:nplot], label="True")
    plt.plot(ts_test.iloc[:nplot], y_pred[:nplot], label="Pred")
    plt.legend(); plt.title(f"{model_name} Actual vs Pred (sample {nplot})")
    plt.xticks(rotation=30)
    plt.savefig(os.path.join(ARTIFACTS, f"{model_name}_actual_vs_pred_timeseries.png")); plt.close()

def save_metrics(all_results: Dict[str, EvalResult]):
    ensure_dir(ARTIFACTS)
    out = {k:v.as_dict() for k,v in all_results.items()}
    with open(os.path.join(ARTIFACTS, "metrics_all.json"), "w") as f:
        json.dump(out, f, indent=2)

# ---------------------------
# Permutation importance (post-fit)
# ---------------------------
def compute_and_save_permutation(pipe, X_test, y_test, numeric, categorical, model_name):
    from sklearn.inspection import permutation_importance
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import os

    print(f"Computing permutation importance for {model_name} ...")

    r = permutation_importance(
        pipe, X_test, y_test,
        n_repeats=10, random_state=42, n_jobs=-1
    )

    # 1) Locate the preprocessor step robustly
    pre = pipe.named_steps.get("pre") or pipe.named_steps.get("preprocessor")
    if pre is None:
        raise KeyError("Could not find preprocessing step. Expected 'pre' or 'preprocessor' in pipeline named_steps.")

    # 2) Numeric feature names (as passed in)
    num_names = list(numeric)

    # 3) Recover OHE-expanded categorical names robustly
    cat_names_expanded = []
    if "cat" in pre.named_transformers_:
        cat_trans = pre.named_transformers_["cat"]
        # Try to find OneHotEncoder inside the categorical pipeline
        ohe = None
        if hasattr(cat_trans, "named_steps") and "ohe" in cat_trans.named_steps:
            ohe = cat_trans.named_steps["ohe"]
        else:
            # Search for OneHotEncoder instance
            for step_name, step_obj in getattr(cat_trans, "named_steps", {}).items():
                from sklearn.preprocessing import OneHotEncoder
                if isinstance(step_obj, OneHotEncoder):
                    ohe = step_obj
                    break
        if ohe is not None:
            try:
                cat_names_expanded = list(ohe.get_feature_names_out(categorical))
            except Exception:
                # Fallback if feature names cannot be extracted
                cat_names_expanded = [f"cat_{i}" for i in range(len(ohe.categories_))]
        else:
            # No OHE found; fallback to original categorical names
            cat_names_expanded = list(categorical)
    else:
        # No categorical transformer present
        cat_names_expanded = []

    feat_names = num_names + cat_names_expanded

    # Align to permutation importance length (safety)
    n_importances = len(r.importances_mean)
    if len(feat_names) != n_importances:
        # Fallback: pad or trim to match lengths
        if len(feat_names) < n_importances:
            feat_names = feat_names + [f"feat_{i}" for i in range(n_importances - len(feat_names))]
        else:
            feat_names = feat_names[:n_importances]

    # Save to CSV
    dfimp = pd.DataFrame({
        "feature": feat_names,
        "importance_mean": r.importances_mean,
        "importance_std": r.importances_std
    }).sort_values("importance_mean", ascending=False)

    os.makedirs(ARTIFACTS, exist_ok=True)
    dfimp.to_csv(os.path.join(ARTIFACTS, f"{model_name}_permutation_importance.csv"), index=False)

    # Save plot
    plt.figure(figsize=(8, 6))
    topk = min(20, len(dfimp))
    sns.barplot(data=dfimp.head(topk), x="importance_mean", y="feature")
    plt.title(f"Top {topk} Features - {model_name} Permutation Importance")
    plt.tight_layout()
    plt.savefig(os.path.join(ARTIFACTS, f"{model_name}_permutation_importance.png"))
    plt.close()

    print(f"Permutation importance saved for {model_name}.")


# ---------------------------
# Main run
# ---------------------------
def main():
    ensure_dir(ARTIFACTS)
    print("Loading dataset...")
    raw = load_dataset(DATASET)
    df = create_time_features(raw, kind=DATASET)

    # Add lag & rolling features (may drop early rows)
    if DATASET=="hour":
        df = add_lag_and_roll_features(df, lags=LAGS, roll_windows=ROLL_WINDOWS)
    else:
        # daily: use only lag 1
        df = add_lag_and_roll_features(df, lags=[1], roll_windows=[7])

    print("Data after FE shape:", df.shape)
    eda_plots(df)

    # Optionally transform target
    if USE_LOG_TARGET:
        df["y"] = np.log1p(df["cnt"])
    else:
        df["y"] = df["cnt"]

    # Chronological split
    n = len(df)
    split_idx = int((1-TEST_SIZE) * n)
    train = df.iloc[:split_idx].copy()
    test  = df.iloc[split_idx:].copy()
    print(f"Train rows: {len(train)}, Test rows: {len(test)}")

    numeric, categorical = get_feature_lists(df)
    X_train = train[numeric + categorical]
    y_train = train["y"].values if USE_LOG_TARGET else train["cnt"].values
    X_test  = test[numeric + categorical]
    y_test  = test["y"].values if USE_LOG_TARGET else test["cnt"].values

    print("Numeric features:", numeric)
    print("Categorical features:", categorical)

    pre = build_preprocessor(numeric, categorical)
    fitted = fit_and_tune(X_train, y_train, pre)

    # Evaluate all fitted models and pick best by test RMSE
    all_results = {}
    preds = {}
    for name, pipe in fitted.items():
        res, ytr_pred, yte_pred = evaluate_model(pipe, X_train, y_train, X_test, y_test, use_log=USE_LOG_TARGET)
        all_results[name] = res
        preds[name] = yte_pred
        # save predictions & diagnostic plots per model
        # use test timestamp for plotting
        plot_predictions_and_residuals(test["timestamp"], y_test, yte_pred, name)

    # pick best
    best_name = min(all_results.keys(), key=lambda k: all_results[k].test_rmse)
    print("\nModel results:")
    for k,v in all_results.items():
        print(f"{k}: Test RMSE={v.test_rmse:.3f}, R2={v.test_r2:.3f}")

    print(f"\nBest by Test RMSE: {best_name}")
    save_metrics(all_results)

    # Save test predictions for best model
    best_pipe = fitted[best_name]
    best_pred = preds[best_name]
    out_df = pd.DataFrame({
        "timestamp": test["timestamp"].values,
        "y_true": test["cnt"].values,
        "y_pred": np.round(best_pred, 3)
    })
    out_df.to_csv(os.path.join(ARTIFACTS, "test_predictions_best.csv"), index=False)
    # Save model
    joblib.dump(best_pipe, os.path.join(ARTIFACTS, "best_pipe.joblib"))

    # Permutation importance for best model
    compute_and_save_permutation(best_pipe, X_test, test["cnt"].values, numeric, categorical, best_name)

    print("\nAll artifacts saved to:", os.path.abspath(ARTIFACTS))

if __name__=="__main__":
    main()


Loading dataset...
Loaded hour.csv shape=(17379, 17)
Data after FE shape: (17355, 24)

--- EDA plots saving to artifacts/ ---
EDA plots saved.
Train rows: 13884, Test rows: 3471
Numeric features: ['temp', 'atemp', 'hum', 'windspeed', 'hr_sin', 'hr_cos', 'doy_sin', 'doy_cos', 'lag_1', 'lag_24', 'roll_mean_3', 'roll_mean_24']
Categorical features: ['season', 'mnth', 'hr', 'weekday', 'workingday', 'holiday', 'weathersit', 'yr', 'is_weekend']

Tuning Ridge ...
 -> best params: {'model__alpha': np.float64(0.001)}

Tuning Lasso ...
 -> best params: {'model__alpha': np.float64(0.0035564803062231283)}

Tuning HistGB ...
 -> best params: {'model__min_samples_leaf': 5, 'model__max_leaf_nodes': 15, 'model__max_iter': 400, 'model__learning_rate': 0.1}

Model results:
Ridge: Test RMSE=75.706, R2=0.882
Lasso: Test RMSE=75.703, R2=0.882
HistGB: Test RMSE=45.619, R2=0.957

Best by Test RMSE: HistGB
Computing permutation importance for HistGB ...
Permutation importance saved for HistGB.

All artifacts 